In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

In [2]:
from ESRNN.m4_data import *
from ESRNN.utils_evaluation import evaluate_prediction_owa
from ESRNN.utils_visualization import plot_grid_prediction

In [3]:
seq_length = 24*7
target_seq_length = 24*2
input_size = 1
hidden_size = 10
num_layers = 1
output_size = 1

In [4]:
X_train_df, y_train_df, X_test_df, y_test_df = prepare_m4_data(dataset_name="Hourly",
                                                               directory="../data/M4",
                                                               num_obs=414)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
def normalize_data(data):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_data = scaler.fit_transform(data.reshape(-1, 1))
    scaled_data = scaled_data.flatten()

    return scaled_data, scaler
    

In [32]:
def create_sequence(df, unique_id, seq_length):
    # Xs, Ys = [], []
    sequence_dict = {}
    grouped = df.groupby(unique_id)

    for group_id, group in grouped:
        data = group['y'].values
        data, scaler = normalize_data(data)
        X, Y = [], []
        for i in range(len(data) - seq_length):
            X.append(data[i:(i + seq_length)])
            Y.append(data[i + seq_length])
        sequence_dict[group_id] = {'X' : np.array(X), 'y': np.array(Y), 'scaler' : scaler}

    return sequence_dict

In [33]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        lstm_out, _ = self.lstm(x, (h0, c0))
        out = self.fc(lstm_out[:, -1, :])
        return out

In [34]:
model = LSTM_Model(input_size, hidden_size, num_layers, output_size)
model.to(device)

LSTM_Model(
  (lstm): LSTM(1, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=1, bias=True)
)

In [35]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [36]:
def train_model(model,
                criterion,
                optimizer,
                X_train,
                y_train,
                target_seq_length = target_seq_length,
                batch_size = 64,
                epochs=100):
    
    dataset = TensorDataset(X_train, y_train)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):

        model.train()

        for batch_idx, (sequences, targets) in enumerate(data_loader):

            sequences, targets = sequences.to(device), targets.to(device)

            # put default model grads to zero
            optimizer.zero_grad()

            # predict the output
            pred = model(sequences)

            # calculate the loss
            loss = criterion(pred, targets)

            # backpropagate the error
            loss.backward()

            # update the model parameters
            optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

    # generating forecasts
    model.eval()
    last_sequence = X_train[-1:].to(device)
    forecast_seq = torch.Tensor().to(device)

    for _ in range(target_seq_length):
        with torch.no_grad():
            
            next_step_forecast = model(last_sequence)

            forecast_seq = torch.cat((forecast_seq, next_step_forecast[:, -1:]), dim=1)

            last_sequence = torch.cat((last_sequence[:, 1:, :], next_step_forecast[:, -1:].unsqueeze(-1)), dim=1)
    
    return model, forecast_seq

In [37]:
sequences_dict = create_sequence(y_train_df, 'unique_id', seq_length)

In [38]:
y_hat_df_lstm = X_test_df.copy().rename(columns={'x' : 'y_hat'})
y_hat_df_lstm['y_hat'] = pd.Series(dtype='float64')

all_forecast_seq_descaled = []

for series_id in sequences_dict.keys():

    print(f'Series {series_id}')
    
    X_train = torch.from_numpy(sequences_dict[series_id]['X'].astype(np.float32)).unsqueeze(-1)
    y_train = torch.from_numpy(sequences_dict[series_id]['y'].astype(np.float32)).unsqueeze(-1)

    model_lstm, forecast_seq = train_model(model,
                                          criterion=criterion,
                                          optimizer=optimizer,
                                          X_train=X_train,
                                          y_train=y_train,
                                          batch_size=128,
                                          epochs=1000)
    
    forecast_seq_descaled = sequences_dict[series_id]['scaler'].inverse_transform(forecast_seq.cpu().numpy()).flatten()
    # all_forecast_seq_descaled = np.hstack(all_forecast_seq_descaled, forecast_seq_descaled)
    all_forecast_seq_descaled.append(forecast_seq_descaled)

all_forecast_seq_descaled = np.hstack(all_forecast_seq_descaled)

y_hat_df_lstm['y_hat'] = all_forecast_seq_descaled


Series H1
Epoch [100/1000], Loss: 0.007330859545618296
Epoch [200/1000], Loss: 0.005395553074777126
Epoch [300/1000], Loss: 0.0033181419130414724
Epoch [400/1000], Loss: 0.002520756097510457
Epoch [500/1000], Loss: 0.00205148640088737
Epoch [600/1000], Loss: 0.0017469633603468537
Epoch [700/1000], Loss: 0.0014947050949558616
Epoch [800/1000], Loss: 0.0012484918115660548
Epoch [900/1000], Loss: 0.001089887460693717
Epoch [1000/1000], Loss: 0.0010201043915003538
Series H10
Epoch [100/1000], Loss: 0.001508890651166439
Epoch [200/1000], Loss: 0.0014301635092124343
Epoch [300/1000], Loss: 0.0013720359420403838
Epoch [400/1000], Loss: 0.0013913629809394479
Epoch [500/1000], Loss: 0.001262700418010354
Epoch [600/1000], Loss: 0.0012152548879384995
Epoch [700/1000], Loss: 0.001120752072893083
Epoch [800/1000], Loss: 0.0011342898942530155
Epoch [900/1000], Loss: 0.0011228708317503333
Epoch [1000/1000], Loss: 0.0010689784539863467
Series H100
Epoch [100/1000], Loss: 0.00915597565472126
Epoch [200

In [39]:
evaluate_prediction_owa(y_hat_df_lstm, y_train_df, X_test_df, y_test_df, naive2_seasonality=24)

===============  Model evaluation  ==============
OWA: 1.698 
SMAPE: 23.879 
MASE: 5.023 


(1.6981861970614656, 5.023374095287388, 23.8787192991257)

In [24]:
y_hat_df_lstm.to_csv('../results/m4/y_hat_df_lstm.csv', index=False)

In [18]:
prediction_length = 24*2

In [19]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

In [20]:
seq_length = 24*7
target_seq_length = prediction_length
input_size = 1
hidden_size = 10
num_layers = 1
output_size = 1
learning_rate = 0.005
epochs = 1500
batch_n = 128

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
def normalize_data(data):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_data = scaler.fit_transform(data.reshape(-1, 1))
    scaled_data = scaled_data.flatten()

    return scaled_data, scaler
    

In [23]:
def create_sequence(df, unique_id, seq_length):
    # Xs, Ys = [], []
    sequence_dict = {}
    grouped = df.groupby(unique_id)

    for group_id, group in grouped:
        data = group['y'].values
        data, scaler = normalize_data(data)
        X, Y = [], []
        for i in range(len(data) - seq_length):
            X.append(data[i:(i + seq_length)])
            Y.append(data[i + seq_length])
        sequence_dict[group_id] = {'X' : np.array(X), 'y': np.array(Y), 'scaler' : scaler}

    return sequence_dict

In [24]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        lstm_out, _ = self.lstm(x, (h0, c0))
        out = self.fc(lstm_out[:, -1, :])
        return out

In [25]:
def train_model(model,
                criterion,
                optimizer,
                X_train,
                y_train,
                target_seq_length = target_seq_length,
                batch_size = 64,
                epochs=100):
    
    dataset = TensorDataset(X_train, y_train)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):

        model.train()

        for batch_idx, (sequences, targets) in enumerate(data_loader):

            sequences, targets = sequences.to(device), targets.to(device)

            # put default model grads to zero
            optimizer.zero_grad()

            # predict the output
            pred = model(sequences)

            # calculate the loss
            loss = criterion(pred, targets)

            # backpropagate the error
            loss.backward()

            # update the model parameters
            optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

    # generating forecasts
    model.eval()
    last_sequence = X_train[-1:].to(device)
    forecast_seq = torch.Tensor().to(device)

    for _ in range(target_seq_length):
        with torch.no_grad():
            
            next_step_forecast = model(last_sequence)

            forecast_seq = torch.cat((forecast_seq, next_step_forecast[:, -1:]), dim=1)

            last_sequence = torch.cat((last_sequence[:, 1:, :], next_step_forecast[:, -1:].unsqueeze(-1)), dim=1)
    
    return model, forecast_seq

In [26]:
sequences_dict = create_sequence(y_train_df, 'unique_id', seq_length)

In [27]:
y_hat_df_lstm = X_test_df.copy().rename(columns={'x' : 'y_hat'})
y_hat_df_lstm['y_hat'] = pd.Series(dtype='float64')


all_forecast_seq_descaled = []

for series_id in sequences_dict.keys():

    print(f'Series {series_id}')
    
    X_train = torch.from_numpy(sequences_dict[series_id]['X'].astype(np.float32)).unsqueeze(-1)
    y_train = torch.from_numpy(sequences_dict[series_id]['y'].astype(np.float32)).unsqueeze(-1)

    model = LSTM_Model(input_size, hidden_size, num_layers, output_size)
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    model_gru, forecast_seq = train_model(model,
                                          criterion=criterion,
                                          optimizer=optimizer,
                                          X_train=X_train,
                                          y_train=y_train,
                                          batch_size=batch_n,
                                          epochs=epochs)
    
    forecast_seq_descaled = sequences_dict[series_id]['scaler'].inverse_transform(forecast_seq.cpu().numpy()).flatten()
    # all_forecast_seq_descaled = np.hstack(all_forecast_seq_descaled, forecast_seq_descaled)
    all_forecast_seq_descaled.append(forecast_seq_descaled)

all_forecast_seq_descaled = np.hstack(all_forecast_seq_descaled)

y_hat_df_lstm['y_hat'] = all_forecast_seq_descaled


Series H1
Epoch [100/1500], Loss: 0.0016405901405960321
Epoch [200/1500], Loss: 0.0010121027007699013
Epoch [300/1500], Loss: 0.0009138003224506974
Epoch [400/1500], Loss: 0.0008334973244927824
Epoch [500/1500], Loss: 0.0007170981843955815
Epoch [600/1500], Loss: 0.0006432684604078531
Epoch [700/1500], Loss: 0.0005849121953360736
Epoch [800/1500], Loss: 0.0005186604685150087
Epoch [900/1500], Loss: 0.00047107841237448156
Epoch [1000/1500], Loss: 0.00041746190981939435
Epoch [1100/1500], Loss: 0.000563307898119092
Epoch [1200/1500], Loss: 0.00034894910641014576
Epoch [1300/1500], Loss: 0.00036756761255674064
Epoch [1400/1500], Loss: 0.00033279889612458646
Epoch [1500/1500], Loss: 0.0002886492293328047
Series H10
Epoch [100/1500], Loss: 0.0010367133654654026
Epoch [200/1500], Loss: 0.0018787343287840486
Epoch [300/1500], Loss: 0.0010441072518005967
Epoch [400/1500], Loss: 0.0006367110763676465
Epoch [500/1500], Loss: 0.0004538374487310648
Epoch [600/1500], Loss: 0.00256429985165596
Epoch

In [28]:
y_hat_df_lstm.reset_index(drop=True, inplace=True)

In [32]:
evaluate_prediction_owa(y_hat_df_lstm, y_train_df, X_test_df, y_test_df, naive2_seasonality=24)

===============  Model evaluation  ==============
OWA: 1.291 
SMAPE: 19.957 
MASE: 3.584 


(1.2910063368904057, 3.5839191445788883, 19.956829161915195)

In [31]:
y_hat_df_lstm.to_csv('../results/m4/y_hat_df_lstm.csv', index=False)